In [20]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVC

In [21]:
data = pd.read_csv("../data/data_folder/train.csv")

# EDA

In [22]:
data.head(5)

,Severity,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Accident_Type_Code,Max_Elevation,Violations,Adverse_Weather_Metric,Accident_ID
0,Minor_Damage_And_Injuries,49.223744,14,22,71.285324,0.272118,78.04,2,31335.476824,3,0.424352,7570
1,Minor_Damage_And_Injuries,62.465753,10,27,72.288058,0.423939,84.54,2,26024.711057,2,0.352350,12128
2,Significant_Damage_And_Fatalities,63.059361,13,16,66.362808,0.322604,78.86,7,39269.053927,3,0.003364,2181
3,Significant_Damage_And_Serious_Injuries,48.082192,11,9,74.703737,0.337029,81.79,3,42771.499200,1,0.211728,5946
4,Significant_Damage_And_Fatalities,26.484018,13,25,47.948952,0.541140,77.16,3,35509.228515,2,0.176883,9054


In [23]:
Minor_Damage_And_Injuries = data.loc[data['Severity'] == 'Minor_Damage_And_Injuries']
Significant_Damage_And_Fatalities = data.loc[data['Severity'] == 'Significant_Damage_And_Fatalities']
Significant_Damage_And_Serious_Injuries = data.loc[data['Severity'] == 'Significant_Damage_And_Serious_Injuries']
Highly_Fatal_And_Damaging = data.loc[data['Severity'] == 'Highly_Fatal_And_Damaging']

In [24]:
Minor_Damage_And_Injuries.head(10)

,Severity,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Accident_Type_Code,Max_Elevation,Violations,Adverse_Weather_Metric,Accident_ID
0,Minor_Damage_And_Injuries,49.223744,14,22,71.285324,0.272118,78.04,2,31335.476824,3,0.424352,7570
1,Minor_Damage_And_Injuries,62.465753,10,27,72.288058,0.423939,84.54,2,26024.711057,2,0.352350,12128
5,Minor_Damage_And_Injuries,43.333333,15,0,73.336372,0.432954,78.63,2,29288.448105,1,0.394505,10947
7,Minor_Damage_And_Injuries,24.703196,18,4,60.802188,0.548353,83.48,4,22547.195242,2,0.040419,8008
20,Minor_Damage_And_Injuries,52.465753,13,0,73.245214,0.221991,81.62,2,39835.718310,1,0.538862,10309
21,Minor_Damage_And_Injuries,38.447489,17,5,74.931632,0.326571,79.31,2,37435.572795,2,0.508486,3918
23,Minor_Damage_And_Injuries,43.881279,15,1,68.003646,0.338472,78.97,1,45541.236753,2,1.664864,6630
30,Minor_Damage_And_Injuries,68.767123,8,2,74.658159,0.334505,78.78,5,15868.443939,2,0.010775,5438
31,Minor_Damage_And_Injuries,32.648402,15,1,48.632634,0.431151,78.29,4,35960.976402,2,0.065733,6012
38,Minor_Damage_And_Injuries,75.981735,6,12,67.866910,0.352536,81.26,2,28213.427620,4,0.382453,5152


In [25]:
Significant_Damage_And_Fatalities.head(10)

,Severity,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Accident_Type_Code,Max_Elevation,Violations,Adverse_Weather_Metric,Accident_ID
2,Significant_Damage_And_Fatalities,63.059361,13,16,66.362808,0.322604,78.86,7,39269.053927,3,0.003364,2181
4,Significant_Damage_And_Fatalities,26.484018,13,25,47.948952,0.541140,77.16,3,35509.228515,2,0.176883,9054
13,Significant_Damage_And_Fatalities,63.652968,13,4,78.669098,0.232089,80.78,7,30882.726377,3,0.002567,2451
17,Significant_Damage_And_Fatalities,38.767123,10,4,37.830447,0.600282,80.82,4,35910.200412,1,0.066515,2064
18,Significant_Damage_And_Fatalities,78.219178,8,3,61.485871,0.338111,82.20,2,34354.111221,2,0.462514,2765
19,Significant_Damage_And_Fatalities,20.228311,15,2,41.567912,0.687191,78.37,2,11503.388561,1,0.159784,2938
24,Significant_Damage_And_Fatalities,25.296804,14,5,47.265269,0.594873,76.19,5,49288.635507,3,0.032437,11434
25,Significant_Damage_And_Fatalities,61.826484,13,7,66.134913,0.423217,78.07,6,42607.071097,1,0.010607,10987
26,Significant_Damage_And_Fatalities,59.269406,14,3,61.622607,0.331981,90.87,7,32342.313384,3,0.003022,8830
28,Significant_Damage_And_Fatalities,55.022831,15,9,70.328168,0.512291,77.03,7,28451.014701,0,0.002916,4196


In [26]:
Significant_Damage_And_Serious_Injuries.head(10)

,Severity,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Accident_Type_Code,Max_Elevation,Violations,Adverse_Weather_Metric,Accident_ID
3,Significant_Damage_And_Serious_Injuries,48.082192,11,9,74.703737,0.337029,81.79,3,42771.499200,1,0.211728,5946
9,Significant_Damage_And_Serious_Injuries,52.237443,6,4,57.429353,0.465410,79.56,1,37933.399001,1,1.389295,6324
11,Significant_Damage_And_Serious_Injuries,48.127854,11,2,62.807657,0.329096,77.80,3,22570.030398,4,0.111873,8946
12,Significant_Damage_And_Serious_Injuries,42.876712,12,14,68.960802,0.159604,77.23,3,44697.868740,3,0.222510,3629
34,Significant_Damage_And_Serious_Injuries,54.748858,9,28,60.619872,0.527437,79.42,7,52249.610146,3,0.004603,9173
37,Significant_Damage_And_Serious_Injuries,43.652968,12,2,59.662716,0.422136,81.86,6,37571.905100,0,0.009644,382
44,Significant_Damage_And_Serious_Injuries,41.369863,13,20,61.668186,0.387877,80.35,3,37974.617168,0,0.189279,10051
49,Significant_Damage_And_Serious_Injuries,33.059361,15,15,72.333637,0.442330,83.60,1,40822.785445,2,1.484316,11278
52,Significant_Damage_And_Serious_Injuries,41.735160,13,11,71.877849,0.380304,76.20,6,17443.654867,2,0.004776,1180
54,Significant_Damage_And_Serious_Injuries,30.456621,16,0,76.526892,0.239662,78.84,6,37178.376973,2,0.009164,12089


In [27]:
Highly_Fatal_And_Damaging.head(10)

,Severity,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Accident_Type_Code,Max_Elevation,Violations,Adverse_Weather_Metric,Accident_ID
6,Highly_Fatal_And_Damaging,22.009132,15,10,62.853236,0.314671,88.04,4,32342.109345,0,0.058872,4717
8,Highly_Fatal_And_Damaging,54.840183,5,40,69.234275,0.335587,78.24,2,45751.983413,0,0.619186,8179
10,Highly_Fatal_And_Damaging,36.666667,11,11,80.218778,0.452788,81.15,2,31463.568565,2,0.424912,3789
14,Highly_Fatal_And_Damaging,26.027397,14,2,83.546035,0.362994,81.63,3,18915.509025,2,0.095806,4028
15,Highly_Fatal_And_Damaging,49.589041,7,1,85.551504,0.270314,78.86,3,25083.657909,2,0.123761,10493
16,Highly_Fatal_And_Damaging,10.867580,18,1,68.003646,0.309622,82.08,6,47893.717972,3,0.011242,20
22,Highly_Fatal_And_Damaging,44.840183,8,2,80.583409,0.249759,78.21,4,45783.527661,0,0.083992,4830
27,Highly_Fatal_And_Damaging,24.337900,14,0,70.783956,0.267790,76.75,4,26538.322263,1,0.048626,2237
29,Highly_Fatal_And_Damaging,65.753425,5,20,48.131267,0.638147,79.75,3,39150.672196,1,0.194870,4755
32,Highly_Fatal_And_Damaging,48.356164,7,2,76.754786,0.293034,77.27,4,41509.545282,3,0.075830,8779


In [28]:
data.describe()

,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Accident_Type_Code,Max_Elevation,Violations,Adverse_Weather_Metric,Accident_ID
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000
mean,41.876406,12.931100,6.564300,65.145324,0.381495,79.969331,3.814900,32001.803282,2.01220,0.255635,6266.554200
std,16.138072,3.539803,6.971982,11.882934,0.121301,2.759739,1.902577,9431.995196,1.03998,0.381128,3610.170288
min,0.000000,1.000000,0.000000,0.000000,0.134000,74.740000,1.000000,831.695553,0.00000,0.000316,2.000000
25%,30.593607,11.000000,2.000000,56.927985,0.293665,77.960000,2.000000,25757.636908,1.00000,0.012063,3139.750000
50%,41.278539,13.000000,4.000000,65.587967,0.365879,79.540000,4.000000,32060.336419,2.00000,0.074467,6280.500000
75%,52.511416,15.000000,9.000000,73.336372,0.451346,81.560000,5.000000,38380.641513,3.00000,0.354059,9391.500000
max,100.000000,23.000000,54.000000,100.000000,0.882648,97.510000,7.000000,64297.651218,5.00000,2.365378,12500.000000


In [29]:
data.Severity.unique()

array(['Minor_Damage_And_Injuries', 'Significant_Damage_And_Fatalities',
       'Significant_Damage_And_Serious_Injuries',
       'Highly_Fatal_And_Damaging'], dtype=object)

In [30]:
data.Severity.value_counts(dropna=False)

Highly_Fatal_And_Damaging                  3049
Significant_Damage_And_Serious_Injuries    2729
Minor_Damage_And_Injuries                  2527
Significant_Damage_And_Fatalities          1695
Name: Severity, dtype: int64

In [31]:
data.Accident_Type_Code.unique()

array([2, 7, 3, 4, 1, 6, 5])

In [32]:
data.isnull().values.any()

False

In [33]:
for cols in data.columns:
    print(cols,'  :  ', data[cols].nunique())
    print ('\n')

Severity   :   4


Safety_Score   :   1175


Days_Since_Inspection   :   23


Total_Safety_Complaints   :   44


Control_Metric   :   952


Turbulence_In_gforces   :   1117


Cabin_Temperature   :   950


Accident_Type_Code   :   7


Max_Elevation   :   4191


Violations   :   6


Adverse_Weather_Metric   :   4258


Accident_ID   :   10000




In [34]:
data.Accident_Type_Code.unique()

array([2, 7, 3, 4, 1, 6, 5])

In [35]:
data.Violations.unique()

array([3, 2, 1, 0, 4, 5])

In [36]:
data.Days_Since_Inspection.unique()

array([14, 10, 13, 11, 15, 18,  5,  6, 12,  7,  8, 17,  9, 16, 20, 19, 21,
        3,  4,  1, 23, 22,  2])

In [37]:
data.Total_Safety_Complaints.unique()

array([22, 27, 16,  9, 25,  0, 10,  4, 40, 11,  2, 14,  1,  3,  5,  7, 20,
       28, 12,  8, 15, 18, 35,  6, 17, 13, 19, 31, 33, 26, 29, 24, 21, 30,
       44, 39, 34, 23, 54, 32, 37, 36, 38, 43])

# Data Preparation

In [38]:
def compute_hot_encoding_features(X):
        one_hot_encoding = pd.get_dummies(X[['Accident_Type_Code']])
        
        return one_hot_encoding

In [39]:
data['Accident_Type_Code'] = pd.Categorical(data['Accident_Type_Code'])
# data['Violations'] = pd.Categorical(data['Violations'])
# data['Days_Since_Inspection'] = pd.Categorical(data['Days_Since_Inspection'])

In [40]:
one_hot_encoding = compute_hot_encoding_features(data)
data = pd.concat([data, one_hot_encoding], axis=1)

In [41]:
data.head()

,Severity,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Accident_Type_Code,Max_Elevation,Violations,Adverse_Weather_Metric,Accident_ID,Accident_Type_Code_1,Accident_Type_Code_2,Accident_Type_Code_3,Accident_Type_Code_4,Accident_Type_Code_5,Accident_Type_Code_6,Accident_Type_Code_7
0,Minor_Damage_And_Injuries,49.223744,14,22,71.285324,0.272118,78.04,2,31335.476824,3,0.424352,7570,0,1,0,0,0,0,0
1,Minor_Damage_And_Injuries,62.465753,10,27,72.288058,0.423939,84.54,2,26024.711057,2,0.352350,12128,0,1,0,0,0,0,0
2,Significant_Damage_And_Fatalities,63.059361,13,16,66.362808,0.322604,78.86,7,39269.053927,3,0.003364,2181,0,0,0,0,0,0,1
3,Significant_Damage_And_Serious_Injuries,48.082192,11,9,74.703737,0.337029,81.79,3,42771.499200,1,0.211728,5946,0,0,1,0,0,0,0
4,Significant_Damage_And_Fatalities,26.484018,13,25,47.948952,0.541140,77.16,3,35509.228515,2,0.176883,9054,0,0,1,0,0,0,0


# Model

In [42]:
cols = data.columns

In [43]:
X = data.drop(['Severity', 'Accident_ID','Accident_Type_Code'],axis=1, inplace=False)

In [44]:
Y = data['Severity']

In [45]:
cv = StratifiedKFold(n_splits=10, random_state=123, shuffle=False)

In [46]:
# For RandomForest

# param_grid = { 
#     'n_estimators': [50,100,500,1000],
#     'max_features': [2,4,6,8],
#     'min_samples_split': [3],
#     'max_depth': [10,25,50,100]
    
# }


# CV_rf1 = GridSearchCV(estimator=clf, param_grid=param_grid, cv=cv, verbose=1, n_jobs=-1)
# CV_rf1.fit(X, Y)
# print (f'Best Parameters: {CV_rf1.best_params_}')
# print (f'Best Score: {CV_rf1.best_score_}')

In [47]:
# For SVC

# param_grid = {'C':[1,10,100],'gamma':[1,0.1,0.001], 'kernel':['rbf']}

In [48]:
print(X.shape)
print(Y.shape)

(10000, 16)
(10000,)


In [49]:
clf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=50, max_features=8, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=3,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
# clf = SVC()
# CV_svc = GridSearchCV(estimator=clf, param_grid=param_grid, cv=cv, verbose=1)
# CV_svc.fit(X, Y)
# print (f'Best Parameters: {CV_svc.best_params_}')
# print (f'Best Score: {CV_svc.best_score_}')

for (train, test), i in zip(cv.split(X, Y), range(10)):
    clf.fit(X.iloc[train], Y.iloc[train])
    y_pred = clf.predict(X.iloc[test])
    score = 100*f1_score(Y.iloc[test], y_pred, average='weighted')
    print('F1_score:', score ,'/n')

F1_score: 94.41081308155283 /n
F1_score: 93.90925761538827 /n
F1_score: 93.5036432361633 /n
F1_score: 95.30755060125361 /n
F1_score: 95.20897037344066 /n
F1_score: 94.50034452577573 /n
F1_score: 95.20819794255266 /n
F1_score: 94.09699871072988 /n
F1_score: 96.0019745045519 /n
F1_score: 95.188037517129 /n


In [50]:
# clf.fit(X, Y)

In [51]:
feat_labels = list(X.columns)

In [52]:
# Feature Importance score from Random Forest Model
for feature in zip(feat_labels, clf.feature_importances_):
    print(feature)

('Safety_Score', 0.2859846809783421)
('Days_Since_Inspection', 0.22271203804311138)
('Total_Safety_Complaints', 0.020464445754868214)
('Control_Metric', 0.18142727828040708)
('Turbulence_In_gforces', 0.04627261947463132)
('Cabin_Temperature', 0.030519829803390228)
('Max_Elevation', 0.023279326566457367)
('Violations', 0.009526653237626964)
('Adverse_Weather_Metric', 0.05440130772026369)
('Accident_Type_Code_1', 0.00864253376643562)
('Accident_Type_Code_2', 0.0335813304775526)
('Accident_Type_Code_3', 0.016830012986849736)
('Accident_Type_Code_4', 0.02777766830738701)
('Accident_Type_Code_5', 0.010446080441223965)
('Accident_Type_Code_6', 0.009433453238920805)
('Accident_Type_Code_7', 0.01870074092253187)


# Test & Submission

In [53]:
test_data = pd.read_csv("../data/data_folder/test.csv")
test_data

,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Accident_Type_Code,Max_Elevation,Violations,Adverse_Weather_Metric,Accident_ID
0,19.497717,16,6,72.151322,0.388959,78.32,4,37949.724386,2,0.069692,1
1,58.173516,15,3,64.585232,0.250841,78.60,7,30194.805567,2,0.002777,10
2,33.287671,15,3,64.721969,0.336669,86.96,6,17572.925484,1,0.004316,14
3,3.287671,21,5,66.362808,0.421775,80.86,3,40209.186341,2,0.199990,17
4,10.867580,18,2,56.107566,0.313228,79.22,2,35495.525408,2,0.483696,21
5,44.794521,8,14,52.597995,0.366961,77.08,2,15496.086157,1,0.221040,40
6,62.374429,13,13,66.499544,0.375616,88.48,7,23226.236895,3,0.002196,45
7,14.520548,17,5,66.180492,0.413841,80.09,4,39076.313532,1,0.071707,50
8,0.000000,20,0,67.730173,0.203960,78.37,6,38472.698964,2,0.009371,52
9,45.981735,11,4,66.545123,0.252283,82.69,6,25419.801519,2,0.006051,85


In [54]:
test_data.columns

Index(['Safety_Score', 'Days_Since_Inspection', 'Total_Safety_Complaints',
       'Control_Metric', 'Turbulence_In_gforces', 'Cabin_Temperature',
       'Accident_Type_Code', 'Max_Elevation', 'Violations',
       'Adverse_Weather_Metric', 'Accident_ID'],
      dtype='object')

In [55]:
test_data_set = test_data.drop(['Accident_ID'],axis=1, inplace=False)

In [56]:
test_data_set['Accident_Type_Code'] = pd.Categorical(test_data_set['Accident_Type_Code'])
# test_data_set['Violations'] = pd.Categorical(test_data_set['Violations'])
# test_data_set['Days_Since_Inspection'] = pd.Categorical(test_data_set['Days_Since_Inspection'])

In [57]:
one_hot_encoding = compute_hot_encoding_features(test_data_set)
test_data_set = pd.concat([test_data_set, one_hot_encoding], axis=1)

In [58]:
test_data_set = test_data_set.drop(['Accident_Type_Code'],axis=1, inplace=False)

In [59]:
test_data_set

,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Max_Elevation,Violations,Adverse_Weather_Metric,Accident_Type_Code_1,Accident_Type_Code_2,Accident_Type_Code_3,Accident_Type_Code_4,Accident_Type_Code_5,Accident_Type_Code_6,Accident_Type_Code_7
0,19.497717,16,6,72.151322,0.388959,78.32,37949.724386,2,0.069692,0,0,0,1,0,0,0
1,58.173516,15,3,64.585232,0.250841,78.60,30194.805567,2,0.002777,0,0,0,0,0,0,1
2,33.287671,15,3,64.721969,0.336669,86.96,17572.925484,1,0.004316,0,0,0,0,0,1,0
3,3.287671,21,5,66.362808,0.421775,80.86,40209.186341,2,0.199990,0,0,1,0,0,0,0
4,10.867580,18,2,56.107566,0.313228,79.22,35495.525408,2,0.483696,0,1,0,0,0,0,0
5,44.794521,8,14,52.597995,0.366961,77.08,15496.086157,1,0.221040,0,1,0,0,0,0,0
6,62.374429,13,13,66.499544,0.375616,88.48,23226.236895,3,0.002196,0,0,0,0,0,0,1
7,14.520548,17,5,66.180492,0.413841,80.09,39076.313532,1,0.071707,0,0,0,1,0,0,0
8,0.000000,20,0,67.730173,0.203960,78.37,38472.698964,2,0.009371,0,0,0,0,0,1,0
9,45.981735,11,4,66.545123,0.252283,82.69,25419.801519,2,0.006051,0,0,0,0,0,1,0


In [60]:
test_data['Severity'] = clf.predict(test_data_set)

In [61]:
test_data.head()

,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Accident_Type_Code,Max_Elevation,Violations,Adverse_Weather_Metric,Accident_ID,Severity
0,19.497717,16,6,72.151322,0.388959,78.32,4,37949.724386,2,0.069692,1,Highly_Fatal_And_Damaging
1,58.173516,15,3,64.585232,0.250841,78.60,7,30194.805567,2,0.002777,10,Significant_Damage_And_Fatalities
2,33.287671,15,3,64.721969,0.336669,86.96,6,17572.925484,1,0.004316,14,Significant_Damage_And_Serious_Injuries
3,3.287671,21,5,66.362808,0.421775,80.86,3,40209.186341,2,0.199990,17,Highly_Fatal_And_Damaging
4,10.867580,18,2,56.107566,0.313228,79.22,2,35495.525408,2,0.483696,21,Significant_Damage_And_Fatalities


In [62]:
submission = test_data[['Accident_ID','Severity']]

In [63]:
submission.to_csv( "../data/data_folder/submission.csv",index=False)